In [209]:
import numpy as np
import xgboost as xgb
import pandas as pd
from matplotlib import pyplot as plt
from scipy import stats
from sklearn.model_selection import RandomizedSearchCV, KFold
from sklearn.metrics import roc_auc_score, average_precision_score, f1_score
import shap
import random

In [221]:
load_path = "/Volumes/chip-lacava/Groups/BCH-ED/reprocessing/prediction/"

#Load the Shapley values
shap_values = pd.read_csv(load_path+"time_based_shap_values.csv")
shap_values




,csn,snapshot,snapshot.1,sex_f,complaint_contains_abdominal_pain,complaint_contains_assault,complaint_contains_allergic_reaction,complaint_contains_altered_mental_status,complaint_contains_asthma_or_wheezing,complaint_contains_bites_or_stings,...,max_hr,max_pain,max_rr,max_sbp,max_sp_o2,min_hr,min_pain,min_rr,min_sbp,min_sp_o2
0,6140116428,0,0.165662,0.017532,0.036082,0.002459,0.017015,0.0,0.003462,0.003030,...,0.068133,-0.000076,0.147449,-0.000122,-0.000095,0.029324,0.021847,-0.000978,0.0,-0.055288
1,6138671788,108,-0.545355,0.019270,0.050399,0.002293,0.011743,0.0,0.003580,-0.000759,...,-0.071186,-0.000621,-0.058749,-0.000122,0.000195,-0.047108,0.015828,-0.004094,0.0,-0.067446
2,6139020185,6060,0.409570,-0.091022,0.040818,0.001954,0.017666,0.0,0.004893,0.002859,...,0.082593,0.000155,-0.002810,-0.000122,0.000285,-0.052696,-0.021575,0.029060,0.0,0.000929
3,6139539782,180,-0.201735,0.016747,0.041643,0.001802,0.012412,0.0,0.005394,0.002233,...,-0.090711,0.005444,-0.039768,-0.000122,-0.000209,-0.055742,-0.032779,-0.007689,0.0,-0.042849
4,6138231604,120,-0.172951,-0.032773,0.101282,0.002177,0.013481,0.0,0.010703,0.002511,...,0.026294,-0.000042,-0.013519,-0.000122,0.000287,-0.092082,-0.009642,-0.004783,0.0,-0.083084
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,6140490133,4380,0.959073,-0.005863,0.044379,0.001580,0.019283,0.0,0.006537,0.002322,...,-0.068118,-0.000219,0.038159,-0.000122,0.000283,-0.018420,0.002117,-0.000865,0.0,-0.023856
996,6139285324,172,-0.542670,-0.019054,0.023435,0.002406,0.011500,0.0,0.006407,0.003316,...,0.044218,-0.000279,0.013043,-0.000122,-0.000604,0.059605,0.015039,-0.001604,0.0,-0.007279
997,6139265489,420,0.110758,-0.035184,0.103159,0.002208,0.010572,0.0,0.005824,-0.002787,...,-0.118537,0.000515,-0.085293,-0.000448,0.000382,-0.054749,0.035216,-0.007110,0.0,-0.088319
998,6139463104,120,-0.233349,-0.021707,0.039726,0.002419,0.016666,0.0,0.003653,0.002825,...,-0.151907,-0.000153,-0.052900,-0.000122,0.000204,-0.036948,-0.092709,-0.003125,0.0,-0.072568


In [223]:
#Rename columns such that 'timepoint' is the actual value of the timepoint and snapshot is its shapley value


#Load the test data
test_data = pd.read_csv(load_path+"test-data.csv", index_col=0)

#Create IDs for linking
shap_values['id'] = list(range(1, len(shap_values)+1))

#Select only timepoints in this data which have been used to create Shapley values
test_data = pd.merge(test_data, shap_values.loc[:, ['id', 'csn', 'snapshot']], on=['csn', 'snapshot'], how='inner').sort_values(by="id")
shap_values = shap_values.rename(columns={'snapshot': 'timepoint', 'snapshot.1':'snapshot'})

    

In [225]:
#Align the dataframes: exclude CSN and timepoint
shap_cols = shap_values.columns.to_list()
test_cols = test_data.columns.to_list()

print(np.setdiff1d(test_cols, shap_cols))
print(np.setdiff1d(shap_cols, test_cols))
print(len(np.setdiff1d(test_cols, shap_cols)))


shap_cols.remove("csn")
shap_cols.remove("timepoint")
shap_cols.remove("id")


test_cols.remove("csn")
test_cols.remove("is_admitted")
test_cols.remove("V1")




test_data = test_data[shap_cols]
shap_values = shap_values[shap_cols]

test_data

['V1' 'is_admitted']
['timepoint']
2


,snapshot,sex_f,complaint_contains_abdominal_pain,complaint_contains_assault,complaint_contains_allergic_reaction,complaint_contains_altered_mental_status,complaint_contains_asthma_or_wheezing,complaint_contains_bites_or_stings,complaint_contains_burn,complaint_contains_cardiac,...,max_hr,max_pain,max_rr,max_sbp,max_sp_o2,min_hr,min_pain,min_rr,min_sbp,min_sp_o2
212,0,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
308,108,0,0,0,0,0,0,0,0,0,...,-1.348678,6.0,-0.576340,0.0,100.0,-0.744447,0.0,-0.106105,0.0,100.0
123,6060,1,0,0,0,0,0,0,0,0,...,1.292774,6.0,-0.184424,0.0,100.0,-0.817504,0.0,-1.771594,0.0,98.0
169,180,0,0,0,0,0,0,0,0,0,...,-1.886421,5.0,-0.723107,0.0,98.0,-1.354478,3.0,-0.451109,0.0,98.0
651,120,1,0,0,0,0,0,0,0,0,...,2.763580,NaN,-0.254159,NaN,100.0,1.763436,NaN,-0.409495,NaN,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
478,4380,0,0,0,0,0,0,0,0,0,...,-0.624394,NaN,0.171995,0.0,100.0,-0.817504,NaN,0.039440,0.0,98.0
691,172,1,0,0,0,0,0,0,0,0,...,1.974050,NaN,0.767788,0.0,98.0,1.492680,NaN,2.065147,0.0,97.0
363,420,1,0,0,0,0,0,0,0,0,...,-1.671551,7.0,-0.723107,0.0,100.0,-1.076444,5.0,-0.451109,0.0,100.0
103,120,0,0,0,0,0,0,0,0,0,...,-1.080658,0.0,-0.389235,0.0,100.0,-0.311850,0.0,0.357477,0.0,100.0


In [227]:
print(shap_values.shape)
print(test_data.shape)

(1000, 248)
(1000, 248)


In [243]:
feature_names = shap_values.columns.to_list() 


#Print top features.
mean_abs_shap = np.abs(shap_values).mean(0)
feature_order = np.argsort(mean_abs_shap)[::-1]
ordered_feature_names = [feature_names[i] for i in feature_order]
print(ordered_feature_names[0:24])


feature_name_dict = {"triage_acuity_2": "ESI score of 2",
                "triage_acuity_4" : "ESI score of 4",
                "pseudo_nedocs": "Severity-adjusted ED crowding",
                "miles_travelled" : "Distance travelled to hospital",
                "snapshot" : "Time since arrival at ED",
                "complaint_contains_fever": "Complaint relates to fever",
                "complaint_contains_headache": "Complaint relates to headache",
                "complaint_contains_abdominal_pain": "Complaint relates to abdominal pain",
                "triage_acuity_3" : "ESI score of 3",
                "complaint_contains_abdomin" : "Complaint relates to the abdomen",
                "max_hr" : "Maximum recorded heart rate", 
                "max_rr" : "Maximum recorded respiratory rate", 
                "medroute_ns_bolus_iv" : "Has received normal saline intravenously",
                "medroute_d5w_ns_1_000_ml_iv" : "Has received D5W intravenously",
                "ed_arrival_mode_transfer" : "Transferred from another hospital",
                "min_sp_o2": "Minimum recorded oxygen saturation",
                "complaint_contains_vomiting" : "Complaint relates to vomiting", 
                "complaint_contains_lacer" : "Complaint relates to laceration",
                "complaint_contains_cough" : "Complaint relates to cough",
                "complaint_contains_neurologic" : "Complaint relates to neurology",
                "ed_arrival_mode_ems" : "Brought in by EMS",
                "year_of_arrival_2023" : "Visit takes place in 2023",
                "year_of_arrival_2022" : "Visit takes place in 2022",
                "lab_sars_co_v_2_poct" : "Most recent COVID test result",
                "lab_hematocrit" : "Most recent hematocrit result",
                "lab_mcv":"Most recent MCV result",
                "lab_creatinine":"Most recent creatinine result",
                "lab_hemoglobin":"Most recent hemoglobin result",
                "weight":"Weight",
                "min_hr":"Minimum recorded heart rate",
                "lab_glucose":"Most recent glucose result",
                "sdi_score":"Socio-economic deprivation",
                "lab_platelet":"Most recent platelet result",
                "lab_albumin":"Most recent albumin result",
                "lab_potassium":"Most recent potassium result",
                "complaint_contains_extremity":"Extremity complaint",
                "complaint_contains_psych":"Psychiatric complaint",
                "lab_influenza_a_poct": "Most recent Influenza A result",
                "lab_c_reactive_protein": "Most recent CRP result"}


for i in range(len(feature_names)):
    if feature_names[i] in feature_name_dict.keys():
        feature_names[i] = feature_name_dict[feature_names[i]]


explanation = shap.Explanation(
    values=shap_values.values,
    data=test_data.values,
    feature_names=feature_names,
)


fig = plt.figure(figsize=(12, 7))
shap.plots.beeswarm(explanation, max_display=25, color_bar=False, font="Arial")
fig.tight_layout()
fig.savefig(load_path+"overall_SHAP_plot.pdf")
fig.show()

['triage_acuity_4', 'triage_acuity_2', 'miles_travelled', 'snapshot', 'complaint_contains_fever', 'lab_potassium', 'complaint_contains_extremity', 'lab_hematocrit', 'triage_acuity_3', 'pseudo_nedocs', 'ed_arrival_mode_transfer', 'complaint_contains_psych', 'weight', 'min_sp_o2', 'lab_influenza_a_poct', 'complaint_contains_abdominal_pain', 'medroute_d5w_ns_1_000_ml_iv', 'complaint_contains_vomiting', 'ed_arrival_mode_ems', 'lab_c_reactive_protein', 'max_hr', 'year_of_arrival_2022', 'year_of_arrival_2023', 'lab_hemoglobin']


TypeError: beeswarm() got an unexpected keyword argument 'font'

<Figure size 1200x700 with 0 Axes>